<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_3_Agentic_RAG/Agentic_RAG_with_Semantic_Cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic RAG with Semantic Cache

This notebook combines two powerful concepts:

- **Semantic Cache** — A FAISS-backed cache that stores previous query embeddings and their answers. When a new query is semantically similar to a cached one, the stored answer is returned instantly — no LLM or API call needed.
- **Agentic RAG** — An intelligent retrieval system that routes queries to the right knowledge source: OpenAI documentation (via Qdrant), 10-K financial filings (via Qdrant), or live internet search (via SerpApi).

## Architecture

```
User Query
    │
    ▼
┌─────────────────────────────┐
│  Is query time-sensitive?   │  ──YES──▶  Agentic RAG (no caching)
│  (current events, "today",  │
│   live data, etc.)          │
└──────────────┬──────────────┘
               │ NO
               ▼
┌─────────────────────────────┐
│   Semantic Cache Lookup     │  ──HIT──▶  Return cached answer ⚡
│   (FAISS similarity search) │
└──────────────┬──────────────┘
               │ MISS
               ▼
┌─────────────────────────────┐
│      Agentic RAG Router     │
│   (GPT-4o classifies query) │
└──────┬──────────┬───────────┘
       │          │           │
  OPENAI      10K_DOC    INTERNET
  QUERY       QUERY       QUERY
    │            │            │
  Qdrant      Qdrant      SerpApi
  (RAG)       (RAG)      (live web)
       │          │           │
       └──────────┴───────────┘
               │
               ▼
    Store answer in cache 💾
               │
               ▼
          Return answer
```

## Why combine them?

- **Speed**: Cached answers return in milliseconds vs. 2–5 seconds for full RAG.
- **Cost**: Fewer LLM and API calls for repeated or similar questions.
- **Correctness**: Time-sensitive queries (e.g., *"What happened today?"*) always bypass the cache to ensure fresh answers.

## 1. Setup

Install dependencies, clone the course repository (which contains `rag_helpers.py` and the pre-built Qdrant vector database), and import the helper module.

In [1]:
!pip install -U faiss-cpu sentence_transformers transformers openai qdrant_client python-dotenv nest_asyncio -q

import os, sys, nest_asyncio

# ── Colab: clone the course repo if not already present ──────────────────────
try:
    import google.colab
    _REPO = "/content/multi-agent-course"
    if not os.path.exists(_REPO):
        os.system(f"git clone https://github.com/hamzafarooq/multi-agent-course.git")
        print("Repository cloned ✅")
    else:
        print("Repository already present ✅")
    _MODULE_DIR = f"{_REPO}/Module_3_Agentic_RAG"
except ImportError:
    # Running locally — rag_helpers.py lives in Module_3_Agentic_RAG/
    _MODULE_DIR = os.path.dirname(os.path.abspath("__file__"))
    print(f"Running locally — helpers path: {_MODULE_DIR}")

sys.path.insert(0, _MODULE_DIR)
nest_asyncio.apply()  # Required for asyncio.run() inside Jupyter/Colab

from rag_helpers import init_rag, SemanticCaching, agentic_rag_with_cache
print("✅ Helpers imported from rag_helpers.py")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.4/390.4 kB 18.9 MB/s eta 0:00:00
Repository cloned ✅
✅ Helpers imported from rag_helpers.py


## 2. API Keys

**On Google Colab** — store keys in the Secrets panel (`🔑` icon, left sidebar):
| Secret name | Where to get it |
|---|---|
| `SERP_API_KEY` | [serpapi.com](https://serpapi.com) |
| `OPENAI_API_KEY` | [platform.openai.com](https://platform.openai.com) |

**Running locally** — add keys to `Module_3_Agentic_RAG/.env`:
```
serp_api_key=<your_key>
openai_api_key=<your_key>
```
The cell below detects the environment automatically and loads from the right source.

In [2]:
# ── Load API keys ─────────────────────────────────────────────────────────────
try:
    from google.colab import userdata
    serp_api_key   = userdata.get('SERP_API_KEY')
    openai_api_key = userdata.get('OPENAI_API_KEY')
    QDRANT_PATH    = f"{_REPO}/Module_3_Agentic_RAG/Agentic_RAG/qdrant_data"
    print("Colab: credentials loaded from Secrets.")
except ImportError:
    from dotenv import load_dotenv, find_dotenv
    load_dotenv(find_dotenv())
    serp_api_key   = os.getenv("serp_api_key")   or os.getenv("SERP_API_KEY")
    openai_api_key = os.getenv("openai_api_key") or os.getenv("OPENAI_API_KEY")
    QDRANT_PATH    = os.path.join(_MODULE_DIR, "Agentic_RAG", "qdrant_data")
    print("Local: credentials loaded from .env.")

print(f"SerpApi key:    {'✅' if serp_api_key else '❌ MISSING'}")
print(f"OpenAI API key: {'✅' if openai_api_key else '❌ MISSING'}")

# ── Initialise the RAG pipeline (loads models + connects to Qdrant) ───────────
init_rag(openai_api_key=openai_api_key, serp_api_key=serp_api_key, qdrant_path=QDRANT_PATH)

Colab: credentials loaded from Secrets.
SerpApi key:    ✅
OpenAI API key: ✅
Loading Nomic text model for Qdrant retrieval embeddings...


config.json: 0.00B [00:00, ?B/s]

configuration_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- configuration_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

modeling_hf_nomic_bert.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/nomic-ai/nomic-bert-2048:
- modeling_hf_nomic_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

✅ RAG pipeline ready.


## 3. Create the Semantic Cache

`SemanticCaching` is defined in `rag_helpers.py`. It provides:

| Method | Purpose |
|---|---|
| `is_time_sensitive(q)` | Returns `True` for questions with temporal keywords — these always bypass the cache |
| `check_cache(q)` | Embeds the query and runs a FAISS nearest-neighbour search; returns hit/miss + pre-computed embedding |
| `add_to_cache(q, answer, embedding)` | Persists a new entry to FAISS + JSON after a RAG call |

**Similarity threshold** (`threshold=0.2`): distance ≤ 0.2 (Euclidean) counts as a hit. Lower = stricter matching. Try `0.1` for exact-ish matches or `0.35` for a looser hit rate.

In [6]:
# Instantiate the semantic cache
# Set clear_on_init=True to wipe any previously stored entries
cache = SemanticCaching(json_file='rag_cache.json', threshold=0.2, clear_on_init=True)

Loading Nomic embedding model for semantic cache...


Cache embedding model ready.
Semantic cache cleared.


## 4. Agentic RAG Pipeline (from `rag_helpers.py`)

The pipeline components are all defined in `rag_helpers.py` — see the file for full implementations.

| Function | What it does |
|---|---|
| `get_internet_content(query)` | Live Google search via SerpApi |
| `route_query(query)` | GPT-4o classifies into `OPENAI_QUERY`, `10K_DOCUMENT_QUERY`, or `INTERNET_QUERY` |
| `_retrieve_and_respond(query, action)` | Embeds query → searches the right Qdrant collection → generates a cited RAG answer |
| `_run_rag_pipeline(query)` | Orchestrates routing + handler dispatch; returns the answer string |
| **`agentic_rag_with_cache(query, cache)`** | **Main entry point** — applies the cache layer on top of the full pipeline |

**Qdrant collections loaded by `init_rag()`:**
- `opnai_data` — OpenAI Agents documentation  
- `10k_data`   — Uber 2021 & Lyft 2024 10-K filings

## 5. Demo — Semantic Cache + Agentic RAG in Action

`agentic_rag_with_cache(query, cache)` is the single function to call. It handles all routing, retrieval, caching, and display automatically.

### Test queries — three cache paths

| Query | Expected path |
|---|---|
| *"What was Uber's revenue in 2021?"* | Cache MISS → 10K RAG → stored |
| *"How much did Uber earn in 2021?"* | Cache HIT (semantically similar) |
| *"How do I build an agent with the OpenAI Agents SDK?"* | Cache MISS → OpenAI RAG → stored |
| *"What are the best AI tools this week?"* | Time-sensitive → bypass cache → SerpApi |
| *"What is the current stock price of Apple?"* | Time-sensitive → bypass cache → SerpApi |
| *"What are the most popular open-source LLMs?"* | Cache MISS → INTERNET → SerpApi → stored |

In [8]:
result = agentic_rag_with_cache("What was Uber's revenue in 2021?", cache)

👤 Query: What was Uber's revenue in 2021?

❌ Cache MISS — running Agentic RAG pipeline...

📍 Route: 10K_DOCUMENT_QUERY  |  Query about company's financial data

💾 Cached for future similar queries.

🤖 Response:
Uber's revenue in 2021 was $3,208,323,000, or approximately $3.2 billion [1][2].



In [10]:
# Test 2: Cache HIT — semantically similar to Test 1, returns instantly from cache
result = agentic_rag_with_cache("How much did Uber earn in fiscal year 2021?", cache)

👤 Query: How much did Uber earn in fiscal year 2021?

✅ Cache HIT (row 0, similarity: 0.838, 0.124s)

🤖 Response (cached):
Uber's revenue in 2021 was $3,208,323,000, or approximately $3.2 billion [1][2].



In [11]:
# Test 3: Cache MISS — routes to OPENAI_QUERY and stores result
result = agentic_rag_with_cache("How do I build an agent with the OpenAI Agents SDK?", cache)

👤 Query: How do I build an agent with the OpenAI Agents SDK?

❌ Cache MISS — running Agentic RAG pipeline...

📍 Route: OPENAI_QUERY  |  The query is about building an agent using OpenAI's SDK, which relates to OpenAI documentation.

💾 Cached for future similar queries.

🤖 Response:
To build an agent with the OpenAI Agents SDK, follow these steps using the fundamental components of an agent: 

1. **Model**: Choose a large language model (LLM) that will handle reasoning and decision-making for the agent.

2. **Tools**: Define external functions or APIs that the agent can use to take action. Code example: 
   ```python
   weather_agent = Agent(
       name="Weather agent",
       instructions="You are a helpful agent who can talk to users about the weather.",
       tools=[get_weather],
   )
   ```

3. **Instructions**: Provide explicit guidelines and guardrails for how the agent should behave.

Initially, you might consider selecting a robust model for all tasks to establish a performanc

In [12]:
# Test 4: Time-sensitive query — BYPASSES cache, calls Ares API for live answer
result = agentic_rag_with_cache("What are the best AI tools this week?", cache)

👤 Query: What are the best AI tools this week?

⏰ Time-sensitive — bypassing cache for a fresh answer.

📍 Route: INTERNET_QUERY  |  This asks for current AI trends.
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] I tried 70+ best AI tools in 2026
    I went deep into each tool, from image generation to email automation, chatbot building to scheduling assistants.
    Source: https://www.techradar.com/best/best-ai-tools

[2] The best AI productivity tools in 2026
    The list you're about to see contains a collection of great AI productivity tools tested by Zapier's app review team, myself included.
    Source: https://zapier.com/blog/best-ai-productivity-tools/

[3] The 12 Best AI Tools for 2026 (That People Actually Use)
    The 12 Best AI Tools for 2026 (That People Actually Use) · 1. ChatGPT · 2. Gemini · 3. Veo · 4. Claude · 5. Grok · 6. NotebookLM · 7. Lovable.
    Source: https://www.synthesia.io/post/ai-tools



In [13]:
# Test 5: Time-sensitive query — stock price, always fetched live
result = agentic_rag_with_cache("What is the current stock price of Apple?", cache)

👤 Query: What is the current stock price of Apple?

⏰ Time-sensitive — bypassing cache for a fresh answer.

📍 Route: INTERNET_QUERY  |  Real-time data request
Getting your response from the internet 🌐 ...

🤖 Response (live):
[1] Stock Price - Apple - Investor Relations
    Stock Quote: NASDAQ: AAPL ; Day's Open262.60 ; Closing Price260.58 ; Volume30.8 ; Intraday High264.48 ; Intraday Low260.05.
    Source: https://investor.apple.com/stock-price/default.aspx

[2] AAPL: Apple Inc - Stock Price, Quote and News
    Apple Inc AAPL:NASDAQ ; Close. 264.58 quote price arrow up +4.00 (+1.54%) ; Volume. 36,424,718 ; 52 week range. 169.21 - 288.62.
    Source: https://www.cnbc.com/quotes/AAPL

[3] Apple Inc. Stock Quote (U.S.: Nasdaq) - AAPL
    264.49 ; Volume: 42.07M · 65 Day Avg: 48.26M ; 258.16 Day Range 264.75 ; 169.21 52 Week Range 288.62 ...
    Source: https://www.marketwatch.com/investing/stock/aapl?gaa_at=eafs&gaa_n=AWEtsqeMzPxf6OWDbyIh9w5nVWR5xAFmKPsZoBAQUZ9ww7VWYXS7a0jyZbvH&gaa_ts=699

In [14]:
# Test 6: Cache MISS — INTERNET_QUERY, stored after Ares API call
result = agentic_rag_with_cache("What are the most popular open-source LLMs?", cache)

👤 Query: What are the most popular open-source LLMs?

❌ Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY  |  Query not specific to OpenAI.
Getting your response from the internet 🌐 ...

💾 Cached for future similar queries.

🤖 Response:
[1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-sourced model versions released after April 2024.
    Source: https://www.vellum.ai/open-llm-leaderboard

[2] Top 10 open source LLMs for 2025
    Top open source LLMs in 2024 · 1. LLaMA 3 · 2. Google Gemma 2 · 3. Command R+ · 4. Mistral-8x22b · 5. Falcon 2 · 6. Grok 1.5 · 7. Qwen1.5 · 8. BLOOM.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[3] Best Open-source AI models? : r/LocalLLM
    I think Deepseek and Qwen are the way to go for most of them, Janus 7b or stable diffusion or Lumina 2.0 for images, whisper for audio, Deepseek ...
    Source: https://www.reddit

In [15]:
# Test 7: Cache HIT — similar to Test 6
result = agentic_rag_with_cache("Which open-source large language models are most widely used?", cache)

👤 Query: Which open-source large language models are most widely used?

❌ Cache MISS — running Agentic RAG pipeline...

📍 Route: INTERNET_QUERY  |  Query about general LLMs, not specific to OpenAI.
Getting your response from the internet 🌐 ...

💾 Cached for future similar queries.

🤖 Response:
[1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open source LLMs are licensed to be freely used, modified, and distributed by anyone.
    Source: https://www.instaclustr.com/education/open-source-ai/top-10-open-source-llms-for-2025/

[2] The best open source large language model
    The largest open-source models, DeepSeek-V3 and DeepSeek-R1, match GPT-4o and o1-pro, respectively. Newer open source LLMs like Nemotron Llama ...
    Source: https://www.baseten.co/blog/the-best-open-source-large-language-model/

[3] 9 Top Open-Source LLMs for 2026 and Their Uses
    LLM are the foundation models of popular and widely-used chatbots, li

## 6. Inspect the Cache

View all entries currently stored in the semantic cache.

In [16]:
print(f"Total cached entries: {len(cache.cache['questions'])}")
print(f"FAISS index size: {cache.index.ntotal}\n")

for i, (q, a) in enumerate(zip(cache.cache['questions'], cache.cache['response_text'])):
    print(f"[{i}] Q: {q}")
    print(f"    A: {a[:120]}...\n" if len(a) > 120 else f"    A: {a}\n")

Total cached entries: 4
FAISS index size: 4

[0] Q: What was Uber's revenue in 2021?
    A: Uber's revenue in 2021 was $3,208,323,000, or approximately $3.2 billion [1][2].

[1] Q: How do I build an agent with the OpenAI Agents SDK?
    A: To build an agent with the OpenAI Agents SDK, follow these steps using the fundamental components of an agent: 

1. **Mo...

[2] Q: What are the most popular open-source LLMs?
    A: [1] Open LLM Leaderboard 2025
    This LLM leaderboard displays the latest public benchmark performance for SOTA open-so...

[3] Q: Which open-source large language models are most widely used?
    A: [1] Top 10 open source LLMs for 2025
    Unlike proprietary models developed by companies like OpenAI and Google, open s...



## Assignment: Extend the System

Try one or more of these extensions:

1. **Adjustable similarity threshold** — Experiment with `threshold=0.1` (stricter) vs `threshold=0.35` (looser). How does it affect hit rate and answer quality?

2. **Cache TTL (Time-To-Live)** — Add an expiry timestamp to each cache entry. Stale entries (e.g., older than 7 days) should be evicted and re-fetched.

3. **Sub-query division** — Before checking the cache, use a GPT call to split compound questions (e.g., *"What was Uber and Lyft revenue in 2021?"*) into sub-queries. Check and populate the cache per sub-query.

4. **Cache analytics** — Track and display cache hit rate, average latency for hits vs misses, and the most-queried topics over a session.